<a href="https://colab.research.google.com/github/RafaelGallo/LLM/blob/main/LLM_Cluster_kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalando bibliotecas geral
!pip install nltk
!pip install vaderSentiment

In [ ]:
# Importando bibliotecas
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
# English positive phrases
positive_phrases = [
    "I love summer!",
    "This movie is amazing.",
    "I'm very happy today.",
    "I enjoy spending time with my family.",
    "I got a promotion at work, I'm ecstatic!"
]

# English neutral phrases
neutral_phrases = [
    "The weather is nice today.",
    "This is a cup of coffee.",
    "I'm going to work now.",
    "Her phone number is 123-456-7890.",
    "The food is ready."
]

# English negative phrases
negative_phrases = [
    "I'm feeling sad.",
    "I don't like horror movies.",
    "The traffic is terrible today.",
    "I had a very stressful day.",
    "I think I'm sick."
]

# Random samples from each category
phrases = np.random.choice(positive_phrases, 900).tolist() + np.random.choice(neutral_phrases, 900).tolist() + np.random.choice(negative_phrases, 900).tolist()

# Create a DataFrame with the phrases
df = pd.DataFrame({'Frases': phrases})

# Shuffle the rows
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Salvar os dados em um arquivo CSV
df.to_csv('base_de_dados.csv', index=False)

# Visualizar as primeiras linhas do DataFrame
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
# Devisão
train = df['Frases']

In [ ]:
# Vetorizar as frases usando TF-IDF
vectorizer = TfidfVectorizer()

# Treinamento modelo
X_texto = vectorizer.fit_transform(df['Frases'])

In [ ]:
# Determinando o número ideal de clusters usando método Elbow
inertia_values = []
silhouette_scores = []
max_clusters = 15  # Você pode ajustar esse valor conforme necessário

for k in range(2, max_clusters + 1):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_texto)
    inertia_values.append(kmeans.inertia_)
    if k > 1:
        silhouette_scores.append(silhouette_score(X_texto, kmeans.labels_))

# Plotando o gráfico do método Elbow
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(2, max_clusters + 1), inertia_values, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método Elbow')

# Plotando o gráfico da métrica de Silhouette
plt.subplot(1, 2, 2)
plt.plot(range(2, max_clusters + 1), silhouette_scores, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score')

plt.tight_layout()
plt.show()

In [ ]:
# Modelo kmeans

# Usando o número ideal de clusters determinado pelo método Elbow
# Você pode ajustar esse valor conforme necessário
numero_clusters = 4

# Criando o modelo KMeans
modelo_kmeans = KMeans(n_clusters=numero_clusters, random_state=42)
modelo_kmeans.fit(X_texto)

In [ ]:
# Adicionar os rótulos de cluster ao DataFrame
df['cluster'] = modelo_kmeans.labels_
df.head()

In [ ]:
# Obtendo os rótulos dos clusters para as palavras
rotulos_clusters = modelo_kmeans.labels_

# Imprimindo os rótulos dos clusters para cada palavra
for palavra, cluster in zip(X_texto, rotulos_clusters):
    print(f"{palavra}: Cluster {cluster}")

In [ ]:
# PCA
from sklearn.decomposition import PCA

# Reduzindo a dimensionalidade dos dados para 2D usando PCA
pca = PCA(n_components=5, random_state=42)
X_reduced = pca.fit_transform(X_texto.toarray())

# Visualizando modelo
pca

In [ ]:
# Plotando os clusters
plt.figure(figsize=(8, 6))
for cluster_num in range(numero_clusters):
    plt.scatter(X_reduced[rotulos_clusters == cluster_num, 0],
                X_reduced[rotulos_clusters == cluster_num, 1],
                label=f'Cluster {cluster_num}')

plt.title('Clusters palavras com Kmeans')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.grid(False)
plt.show()

In [ ]:
# Vetorizando as frases usando TF-IDF
vectorizer = TfidfVectorizer()

# Treinamento
train = vectorizer.fit_transform(df['Frases'])

# Visualizando modelo
vectorizer

In [ ]:
# Modelo t_SNE
from sklearn.manifold import TSNE

# Aplicando t-SNE para redução de dimensionalidade
tsne = TSNE(n_components=2, random_state=42)

# Treinamento modelo
X_tsne = tsne.fit_transform(train.toarray())

# Visualizando modelo
tsne

In [ ]:
# Criar um gráfico de dispersão com legenda baseada nos clusters
plt.figure(figsize=(10, 8))
for cluster_num in sorted(df['cluster'].unique()):  # Iterar sobre os clusters únicos
    indices = df['cluster'] == cluster_num
    plt.scatter(X_tsne[indices, 0],
                X_tsne[indices, 1],
                alpha=0.5,

                label=f'Cluster {cluster_num}')

plt.title('Visualização de Frases usando t-SNE')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.legend()
plt.show()

# **2) Análise de sentimento com VADER**

In [ ]:
## Visualizando os dados

# Visualizando os 5 primeiros dados
df.head()

In [ ]:
# Visualizando linhas e colunas
df.shape

In [ ]:
# Aplicando análise de sentimendo com vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Inicialize o analisador de sentimentos
analisador = SentimentIntensityAnalyzer()

# Função para calcular o sentimento de uma frase
def analisar_sentimento(frase):
    return analisador.polarity_scores(frase)['compound']

# Aplicar a função de análise de sentimento à coluna 'frases'
df['polaridade'] = df['Frases'].apply(analisar_sentimento)

# Função para categorizar o sentimento
def categorizar_sentimento(compound):
    if compound > 0.05:
        return 'positivo'
    elif compound < -0.05:
        return 'negativo'
    else:
        return 'neutro'

# Função para calcular o sentimento de uma frase e categorizá-la
def analisar_sentimento(frase):
    compound = analisador.polarity_scores(frase)['compound']
    return categorizar_sentimento(compound)

# Aplicar a função de análise de sentimento e categorização à coluna 'frases'
df['sentimento'] = df['Frases'].apply(analisar_sentimento)

# Exibir o DataFrame com as pontuações de sentimento
df.head()

In [ ]:
sns.countplot(x='sentimento', data=df)
plt.title('Distribuição de Sentimentos')
plt.show()

In [ ]:
sns.boxplot(x='sentimento', y='polaridade', hue="sentimento", data=df)
plt.title('Distribuição de Polaridade por Sentimento')
plt.show()

In [ ]:
sns.displot(df['polaridade'], kde=True)
plt.title('Distribuição de Polaridade')
plt.show()

## **3) Sentimento com PCA**

In [ ]:
# Exibir o DataFrame
df.head()

In [ ]:
# Importando biblioteca
from sklearn.feature_extraction.text import TfidfVectorizer

# Vetorizar as frases usando TF-IDF
vectorizer = TfidfVectorizer()

# Treinamento
X_texto = vectorizer.fit_transform(df['Frases'])

# Visualizando
vectorizer

In [ ]:
# Criar DataFrame para as características de texto
X_texto_df = pd.DataFrame(X_texto.toarray(), columns=vectorizer.get_feature_names_out())

# Mapear valores categóricos da coluna 'sentimento' para números
sentimento_numerico = {'positivo': 1, 'neutro': 0, 'negativo': -1}
df['sentimento_numerico'] = df['sentimento'].map(sentimento_numerico)

# Concatenar as características de texto com a coluna 'sentimento_numerico'
X = pd.concat([X_texto_df, df[['sentimento_numerico']]], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Padronizar os dados
scaler = StandardScaler()
X_padronizado = scaler.fit_transform(X)

In [ ]:
# Modelo PCA

# Defina o número de componentes desejado
numero_componentes = 2

# Modelo PCA
pca = PCA(n_components=numero_componentes, random_state=42)

# Treinamento modelo
X_reduzido = pca.fit_transform(X_padronizado)

# Visualizando modelo
pca

In [ ]:
# Criar um DataFrame com os componentes principais e a coluna de sentimento
df_pca = pd.DataFrame(X_reduzido, columns=['Componente 1', 'Componente 2'])
df_pca['sentimento'] = df['sentimento']

# Visualizando modelo PCA
df_pca.head()

In [ ]:
# Plotar o gráfico PCA com legenda
cores = {'positivo': 'blue',
         'neutro': 'gray',
         'negativo': 'red'}

plt.figure(figsize=(10, 8))
for sentimento, cor in cores.items():
    indices = df_pca['sentimento'] == sentimento
    plt.scatter(df_pca.loc[indices, 'Componente 1'],
                df_pca.loc[indices, 'Componente 2'],
                color=cor,
                label=sentimento)

plt.title('PCA com legenda dos sentimentos')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.legend()
plt.show()

In [ ]:
# Modelo 2 - t_SNE

In [ ]:
# Vetorizar as frases usando TF-IDF
vectorizer = TfidfVectorizer()
X_texto = vectorizer.fit_transform(df['Frases'])

In [ ]:
# Aplicar o t-SNE para redução de dimensionalidade
tsne = TSNE(n_components=2, random_state=42)

# Treinamento
X_tsne = tsne.fit_transform(X_padronizado)

# Visualizando modelo
tsne

In [ ]:
# Definir cores para cada categoria de sentimento
cores = {'positivo': 'blue',
         'neutro': 'gray',
         'negativo': 'red'}

# Plotar o gráfico de dispersão com legendas
plt.figure(figsize=(10, 8))
for sentimento, cor in cores.items():
    indices = df['sentimento'] == sentimento
    plt.scatter(X_tsne[indices, 0], X_tsne[indices, 1], alpha=0.5, color=cor, label=sentimento)

plt.title('Visualização de Frases usando t-SNE com Sentimentos')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.legend()
plt.show()

# **4) LLM**

## Modelo 1) EleutherAI/gpt-neo-125M

In [ ]:
# base dados
# Frases com sentimentos positivos
frases_positivas = [
    "I love spending time with my family.",
    "The weather is beautiful today.",
    "I feel happy when I'm with my friends.",
    "I accomplished my goals for today."
]

# Frases com sentimentos negativos
frases_negativas = [
    "I'm feeling sad and lonely.",
    "The news about the accident was heartbreaking.",
    "I failed my exam and I'm disappointed in myself.",
    "I had a terrible day at work."
]

# Frases com sentimentos neutros
frases_neutras = [
    "The cat is sleeping on the couch.",
    "I'm going to the grocery store later.",
    "Today is Monday.",
    "I have to finish my report by tomorrow."
]

# Criar uma lista com todas as frases e seus respectivos sentimentos
frases = []
sentimentos = []

# Adicionar frases positivas
for frase in frases_positivas:
    frases.append(frase)
    sentimentos.append("positivo")

# Adicionar frases negativas
for frase in frases_negativas:
    frases.append(frase)
    sentimentos.append("negativo")

# Adicionar frases neutras
for frase in frases_neutras:
    frases.append(frase)
    sentimentos.append("neutro")

# Embaralhar as frases
indices_embaralhados = list(range(len(frases)))
random.shuffle(indices_embaralhados)

# Criar o DataFrame
df = pd.DataFrame({'Frase': [frases[i] for i in indices_embaralhados]})

# Exibir as primeiras linhas do DataFrame
df.head()

In [ ]:
# Importando biblioteca
from transformers import pipeline
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import pipeline, AutoTokenizer, AutoModel

In [ ]:
# Carregar o modelo e o tokenizador
model_name = "EleutherAI/gpt-neo-125M"

# Carregar o modelo pré-treinado GPT-2
modelo = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
# Tokenização modelo
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Modelo
model = AutoModel.from_pretrained(model_name)

In [ ]:
# Concatenar todas as frases em uma única string
frases_concatenadas = " ".join(df['Frase'].tolist())

In [ ]:
# Tokenizar a string de frases
tokens = tokenizer.encode(frases_concatenadas, return_tensors="pt")

In [ ]:
# Função para gerar texto continuando a sequência de uma única frase
def gerar_texto(frase):
    tokens = tokenizer.encode(frase, return_tensors="pt")
    outputs = modelo.generate(tokens, max_length=20, num_return_sequences=1, early_stopping=True)
    texto_gerado = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return texto_gerado

# Gerar texto para cada frase no DataFrame
for index, row in df.iterrows():
    frase = row['Frase']
    texto_gerado = gerar_texto(frase)
    print(f"Frase original: {frase}")
    print()
    print()
    print(f"Texto gerado: {texto_gerado}\n")

In [ ]:
# Salvando texto em dataframe
# Lista para armazenar os textos gerados
textos_gerados = []

# Gerar texto para cada frase no DataFrame
for index, row in df.iterrows():
    frase = row['Frase']
    texto_gerado = gerar_texto(frase)
    textos_gerados.append(texto_gerado)

# Criar um novo DataFrame com os textos gerados
df_textos_gerados = pd.DataFrame({'Frase Original': df['Frase'],
                                  'Texto Gerado': textos_gerados})

# Salvar o DataFrame em um arquivo CSV
df_textos_gerados.to_csv('textos_gerados.csv', index=False)

In [ ]:
# Visualizando dataframe
df_textos_gerados.head()

In [ ]:
!pip install sentence_transformers

In [ ]:
# Importando biblioteca
from sentence_transformers import SentenceTransformer

# Inicializar o modelo LLM
modelo_llm = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
# Gerar representações vetoriais para as frases
representacoes = modelo_llm.encode(df['Frase'], show_progress_bar=True)

In [ ]:
# Carregar o pipeline para análise de sentimento
analisador_sentimento = pipeline("sentiment-analysis", model="distilbert-base-uncased")

In [ ]:
# Realizar análise de sentimento para cada frase
for frase in frases:
    resultado = analisador_sentimento(frase)
    sentimento = resultado[0]['label']
    if sentimento == 'POSITIVE':
        print(f"Frase: {frase}")
        print("Sentimento: Positivo\n")
    elif sentimento == 'NEGATIVE':
        print(f"Frase: {frase}")
        print("Sentimento: Negativo\n")
    else:
        print(f"Frase: {frase}")
        print("Sentimento: Neutro\n")

In [ ]:
from yellowbrick.cluster import KElbowVisualizer

# Usar o método Elbow para encontrar o número ideal de clusters
visualizador = KElbowVisualizer(KMeans(),
                                k=(2, 10),
                                metric='distortion',
                                timings=False)
# Treinamento
visualizador.fit(representacoes)

# Visualizando
visualizador.show()

In [ ]:
# Obter o número ideal de clusters
numero_clusters = visualizador.elbow_value_

# Criar o modelo KMeans com o número ideal de clusters
modelo_kmeans = KMeans(n_clusters=numero_clusters, random_state=42)

# Treinar o modelo
clusters = modelo_kmeans.fit_predict(representacoes)

# Visualizando
modelo_kmeans

In [ ]:
# Adicionar os clusters de volta ao DataFrame
df['Cluster_LLM'] = clusters

# Visualizando
df.head()

In [ ]:
# Plotar o gráfico
plt.figure(figsize=(10, 6))
plt.scatter(representacoes[:, 0],
            representacoes[:, 1],
            c=clusters,
            cmap='viridis')
plt.title('Clusters LLM')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True)
plt.show()

In [ ]:
# PCA Modelo
# Vetorizar os textos gerados usando TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_textos_gerados['Texto Gerado'])

In [ ]:
# Reduzir a dimensionalidade com PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X.toarray())

In [ ]:
# Adicionar as componentes PCA de volta ao DataFrame
df_textos_gerados['PCA Component 1'] = X_pca[:, 0]
df_textos_gerados['PCA Component 2'] = X_pca[:, 1]

# Plotar o gráfico
plt.figure(figsize=(10, 8))
plt.scatter(df_textos_gerados['PCA Component 1'], df_textos_gerados['PCA Component 2'])
plt.title('PCA Plot dos Textos Gerados')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.show()